# Finding & Enriching Small & Medium Sized Businesses with Sixtyfour

This notebook teaches you on how to use the Sixtyfour API to find and enrich businesses via our api. To get started, go to https://app.sixtyfour.ai and sign up to create an account with an API key.

### Set up

In [7]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

In [9]:
load_dotenv()

True

In [11]:
SIXTYFOUR_API_KEY = os.getenv('SIXTYFOUR_API_KEY')

### Searching for businesses

In [38]:
search_query = 'Find me all photgraphers in La Jolla, California'

Feel free to setup any fields of your choice, the search agent might not have all the information on these in one go, but we will enrich in the next step

In [39]:
fields={
    "company_name":"Name of the company",
    "company_email":"Email for the company",
    "address":"Where are they located",
    "city":"City they are located in",
    "state":"Which state do they operate in",
    "phone_number":"Their phone number",
    "company_linkedin":"Linkedin profile of the company if available",
    "google_profile":"Google business profile if found",
    "website":"Website of the company",
    "key_people":"name of people here seperated by commas"
  }

This step takes a bit as the agent is searching the internet for businesses!

In [40]:
res = requests.post('https://api.sixtyfour.ai/search', headers={
    'x-api-key': SIXTYFOUR_API_KEY,
    "Content-Type": "application/json"
}, json={
    "search_query": search_query,
    "struct": fields
})

In [41]:
df = pd.DataFrame(res.json())

In [42]:
df

,company_name,company_email,address,city,state,phone_number,company_linkedin,google_profile,website,key_people,notes,extra_emails
0,Maria V Photography,,"13634 Vian Rd, Poway, CA 92064",Poway,CA,+18133352197,,,https://www.mariavphotography.com/,,"Photographer. Identifies as women-owned, offer...",
1,Zeena Gregg Photography,,"15604 Riparian Rd, Poway, CA 92064",Poway,CA,+18587800510,,https://www.google.com/maps/place/data=!3m1!4b...,http://www.zeenagregg.com/,,Lead identified as a photographer with a 5-sta...,
2,FarPix,,,,,+18584052023,,,https://www.farpix.com/,,FarPix is a photography service with additiona...,
3,IDM Photos,,"11579 Poblado Rd #b, San Diego, CA 92127",San Diego,CA,+18584927377,,,http://www.idmphotos.com/,,Business is a wedding and portrait photographe...,
4,Fotility Studios,,,,,+18004941405,,https://www.google.com/maps/place/data=!3m1!4b...,http://www.fotilitystudios.com/,,Google listing indicates 24-hour operation eve...,
...,...,...,...,...,...,...,...,...,...,...,...,...
260,Michael Maher Photography,,"7320 Fay Ave, La Jolla, CA 92037",La Jolla,CA,+18584548314,,,http://www.michaelmaherphotography.com/,,This lead is a photography and marketing consu...,
261,LIK Fine Art,,"1205 Prospect St, La Jolla, CA 92037",La Jolla,CA,+18582000990,,https://www.google.com/maps/place/data=!3m1!4b...,https://lik.com/,,LIK Fine Art is an established entity in the a...,
262,Studio Freyja,,"7290 Navajo Rd #107, San Diego, CA 92119",San Diego,CA,+16126554991,,,https://www.studiofreyja.com/,,Company is a photography studio offering vario...,
263,Sundin Photograhy,,,,,+18586102329,,https://www.google.com/maps/place/data=!3m1!4b...,,,This lead is identified as a photographer and ...,


### Next step is to enrich these companies on the account level